## ANN Regression & Functional APIs
ANN model to predict the price of a property.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
house = fetch_california_housing()

In [3]:
house.target_names

['MedHouseVal']

In [4]:
house.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [5]:
from sklearn.model_selection import train_test_split
x_train_full, x_test, y_train_full, y_test = train_test_split(house.data, 
                                                              house.target, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, random_state=42)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

scaler is trained on training data only because we do not want info from test set or validation set to influence model training. scaler will find values from the training data to subtract as a mean & to divide as a std dev. scaler will use these values to standardise validation & test set. 

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
x_train.shape

(11610, 8)

Since this is a regression problem, we will have a single output neuron w/o any activation function. A single neuron because we want a continuous value as o/p.

In [9]:
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, concatenate
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

We need to provide input_shape in the 1st layer of the network.<br>
input_shape = number of independent vars in data

In [10]:
model_A = Sequential()
model_A.add(Dense(30, activation='relu', input_shape = [8]))
# model_A.add(Dense(30, activation='relu', input_shape = x_train.shape[1:]))
model_A.add(Dense(30, activation='relu'))
model_A.add(Dense(1))

model_A.compile(optimizer = SGD(learning_rate = 1e-3),
              loss = "mean_squared_error",
              metrics = ['mae'])
# mae is the mean absolute error = |predicted value - actual value|
# mean_squared_error = mae^2

In [11]:
input_ = Input(shape=x_train.shape[1:])
hidden1 = Dense(30, activation='relu')(input_)
hidden2 = Dense(30, activation='relu')(hidden1)
concat = concatenate([input_,hidden2])
output = Dense(1)(concat)
model_B = Model(inputs=[input_], outputs=[output])

model_B.compile(optimizer = SGD(learning_rate = 1e-3),
                loss = "mean_squared_error",
                metrics = ['mae'])

In [12]:
checkpoint_cb_A = keras.callbacks.ModelCheckpoint("HousePrice_model_A.h5", save_best_only=True)
checkpoint_cb_B = keras.callbacks.ModelCheckpoint("HousePrice_model_B.h5", save_best_only=True)
early_stopping_cb_A = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
early_stopping_cb_B = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

keras stores the weights & biases in memory. if we re-run the model.fit() 2 times, that will be similar to running model.fit() with 40 epochs. in the re-run, it will start with 21st epoch.

In [13]:
history_A = model_A.fit(x_train, y_train, epochs=20, 
                      validation_data=(x_val,y_val),
                      callbacks = [checkpoint_cb_A, early_stopping_cb_A])

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 1.9606 - mae: 1.0113 - val_loss: 0.9678 - val_mae: 0.6920
Epoch 2/20
102/363 [=======>......................] - ETA: 0s - loss: 0.8186 - mae: 0.6780

C:\Users\geetika singh\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


363/363 [==============================] - 1s 2ms/step - loss: 0.7424 - mae: 0.6450 - val_loss: 0.9821 - val_mae: 0.6114
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6489 - mae: 0.5974 - val_loss: 0.6635 - val_mae: 0.5668
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5997 - mae: 0.5700 - val_loss: 0.5415 - val_mae: 0.5438
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5624 - mae: 0.5493 - val_loss: 0.5076 - val_mae: 0.5222
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5312 - mae: 0.5326 - val_loss: 0.5285 - val_mae: 0.5097
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5057 - mae: 0.5175 - val_loss: 0.4910 - val_mae: 0.4959
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4843 - mae: 0.5050 - val_loss: 0.4430 - val_mae: 0.4798
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4663 - mae:

In [15]:
history_B = model_B.fit(x_train, y_train, epochs=20, 
                        validation_data=(x_val, y_val),
                        callbacks = [checkpoint_cb_B, early_stopping_cb_B])

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.6393 - mae: 1.1665 - val_loss: 0.7854 - val_mae: 0.6227
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6627 - mae: 0.5945 - val_loss: 0.6803 - val_mae: 0.5676
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6072 - mae: 0.5657 - val_loss: 0.5677 - val_mae: 0.5396
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5703 - mae: 0.5470 - val_loss: 0.5302 - val_mae: 0.5299
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5465 - mae: 0.5338 - val_loss: 0.5248 - val_mae: 0.5147
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5224 - mae: 0.5242 - val_loss: 0.4770 - val_mae: 0.5018
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5083 - mae: 0.5154 - val_loss: 0.4632 - val_mae: 0.4925
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
pd.DataFrame(history_A.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

this graph is still going down, means that if we run some more epochs, it will further decrease the losses & improve accuracy. 

In [ ]:
pd.DataFrame(history_B.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [14]:
model_A = load_model("HousePrice_model_A.h5")
model_A.evaluate(x_test,y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.3949 - mae: 0.4497


[0.39493411779403687, 0.44973522424697876]

In [16]:
model_B = keras.models.load_model("HousePrice_model_B.h5")
model_B.evaluate(x_test,y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.4271 - mae: 0.4687


[0.42712682485580444, 0.4686586260795593]

In [17]:
y_pred = model_A.predict(x_test[:3])
y_pred

1/1 [==============================] - 0s 88ms/step


array([[0.61997306],
       [1.5508323 ],
       [3.405706  ]], dtype=float32)

In [18]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])

here, normal sequential MLP model is performing better than wide MLP model. 